In [ ]:
%pip install "drawsvg~=2.0"

import csv
import itertools
import math
import os
import random
import drawsvg as dw
import numpy as np

## Define sampling params here

In [3]:
sample_size = 50
angle_variation_max = 20
variation_step = 1
fixed_vars = ["ref_angle"]

w, h = 600, 600

save_path = "../jnd_images_svg/wedges_angle"
os.makedirs(save_path, exist_ok=True)

## Define hyperparam space

In [ ]:
hparam_dict = {
    "line_color": ["gray", "none"],
    "fill_colors": list(itertools.permutations(["red", "green", "blue", "lightblue"], 2)),
    "line_thickness": range(1, 4),
    "shape_location": [0.25, 0.5, 0.75],
    "shape_dist": [0.05, 0.1, 0.15],
    "reference_radius": [40, 50, 60],
    "ref_angle": [30, 60, 90, 120]
}

selected_hparams = [key for key in hparam_dict if key not in fixed_vars]
selected_hparam_vals = [hparam_dict[key] for key in selected_hparams]
fixed_hparam_vals = [hparam_dict[key] for key in fixed_vars]
print(selected_hparams)

all_hparams = list(itertools.product(*selected_hparam_vals))
fixed_hparams_exhaustive = list(itertools.product(*fixed_hparam_vals))

print(len(all_hparams))
print(len(fixed_hparams_exhaustive))

## Sample

In [5]:
random_hparam_configs = random.choices(all_hparams, k=sample_size)

## Generate & save

In [ ]:
def create_sector(origin_x, origin_y, center_x, center_y, radius, start_angle_deg, end_angle_deg, rotation, **kwargs):
    """
    Creates a drawsvg Path object for a circular sector.
    This works for any angle, including those > 180 degrees.
    """
    # Convert angles to radians
    start_rad = math.radians(start_angle_deg)
    end_rad = math.radians(end_angle_deg)

    transform_str = f"translate({center_x}, {center_y}), rotate({rotation})"

    # Calculate start and end points of the arc
    p1_x = origin_x + radius * math.cos(start_rad)
    p1_y = origin_y + radius * math.sin(start_rad)
    p2_x = origin_x + radius * math.cos(end_rad)
    p2_y = origin_y + radius * math.sin(end_rad)

    # Determine the large-arc-flag
    angle_diff = abs(end_angle_deg - start_angle_deg)
    large_arc_flag = 1 if angle_diff > 180 else 0

    # Construct the SVG path data string
    path_data = (
        f"M {origin_x},{origin_y} "  # Move to center
        f"L {p1_x},{p1_y} "          # Line to arc start
        f"A {radius},{radius} 0 {large_arc_flag} 1 {p2_x},{p2_y} " # Arc to end point
        f"Z"                         # Close path back to center
    )

    # Return the Path object
    return dw.Path(d=path_data, transform=transform_str, **kwargs)


oob_count = 0
overlapping_count = 0
total_count = 0
param_d_list = []

# Iterate over randomly selected hparam configurations minus the fixed hparam
for i, config in enumerate(random_hparam_configs):
    config_d = dict(zip(selected_hparams, config))
    
    # Iterate over all values of a fixed variable
    for j, fixed_config in enumerate(fixed_hparams_exhaustive):
        config_d.update(dict(zip(fixed_vars, fixed_config)))
        
        # Iterate over target manipulation (radius)
        for k, angle_diff in enumerate(np.arange(0, angle_variation_max, variation_step)):
            location = config_d["shape_location"]
            ref_r = config_d["reference_radius"]
            shape_dist = config_d["shape_dist"]
            line_color = config_d["line_color"]
            fill_color_1, fill_color_2 = config_d["fill_colors"]
            thickness = config_d["line_thickness"]
            ref_angle = config_d["ref_angle"]
            
            ref_x, ref_y = w*location, h*location
            max_ref_x = ref_x+ref_r
            max_ref_y = ref_y+ref_r

            angle = ref_angle + angle_diff

            # Count cases where shapes go out of bounds
            total_count += 1
            oob = False
            if max_ref_x > w or max_ref_y > h:
                oob = True
                oob_count += 1

            d = dw.Drawing(
                w, h, origin=(0,0),
                context=None,
                animation_config=None,
                id_prefix='d'
            )

            sector = create_sector(0, 0, ref_x, ref_y, ref_r, -ref_angle/2, ref_angle/2, 0, fill=fill_color_1)
            sector2 = create_sector(0, 0, ref_x, ref_y, ref_r, -angle/2, angle/2, ref_angle, fill=fill_color_2)
            d.append(sector)
            d.append(sector2)

            # Outline circle (or not)
            d.append(dw.Circle(ref_x, ref_y, ref_r,
                            fill='none', stroke=line_color))

            display(d)
            idx = f"{i}_{j}_{angle_diff}"
            filename = f"{idx}.svg"
            print(filename)
            d.save_svg(os.path.join(save_path, filename))
        
            param_d = {
                "idx": idx,
                "filename": filename,
                "ref_x": ref_x,
                "ref_y": ref_y,
                "line_color": line_color,
                "fill_color_1": fill_color_1,
                "fill_color_2": fill_color_2,
                "line_thickness": thickness,
                "shape_location": location,
                "shape_dist": shape_dist,
                "reference_radius": ref_r,
                "angle_diff": angle_diff,
            }

            param_d_list.append(param_d)

print("Total # images: ", total_count)
print("Total out of bound images: ", oob_count)
print("Total overlapping: ", overlapping_count)

## Save metadata

In [ ]:
print(param_d_list[-1])

with open(os.path.join(save_path, "wedge_size_jnd_images_svg.csv"), 'w') as wf:
    writer = csv.DictWriter(wf, fieldnames=param_d_list[-1].keys())
    writer.writeheader()
    for d in param_d_list:
        writer.writerow(d)